## Sample Notebook for making predictions of mlcsv_future data

In [1]:
import numpy as np
import pandas as pd
from getpass import getpass
from sqlalchemy import create_engine,Table, MetaData, text, func
from sqlalchemy.orm import sessionmaker
from sqlalchemy.orm import Session
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import session
import sqlalchemy_utils
from sqlalchemy_utils import database_exists, create_database
import psycopg2
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey
from sqlalchemy import inspect
import pyodbc 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
from tensorflow import keras


### Read Test Future data into DataFrame

In [2]:
future_df = pd.read_csv('Resources/mlcsv_future.csv')

In [3]:
future_df

,spkid,full_name,pdes,pha,h,orbit_id,epoch,epoch.mjd,epoch.cal,e,...,condition_code,rms,orbit_id_cad,jd,cd,dist,dist_min,dist_max,v_rel,v_inf
0,1000020,72P/Denning-Fujikawa,72P,NaN,NaN,JPL 21,2456981.5,56981,2014-11-20.0,0.8192,...,1,1.03990,21,2473323.674,10:00.0,0.172866,0.172845,0.172887,17.489102,17.488220
1,1000025,2P/Encke,2P,NaN,NaN,JPL K204/19,2457296.5,57296,2015-10-01.0,0.8483,...,0,0.70018,K204/19,2474742.764,20:00.0,0.194563,0.194558,0.194568,32.478314,32.477892
2,1000027,15P/Finlay,15P,NaN,NaN,JPL K213/12,2457215.5,57215,2015-07-12.0,0.7202,...,0,0.74310,K213/13,2473755.222,20:00.0,0.033432,0.033417,0.033448,12.551198,12.544847
3,1000056,104P/Kowal 2,104P,NaN,NaN,JPL K224/9,2458816.5,58816,2019-11-29.0,0.6765,...,1,0.80948,K224/10,2473780.258,11:00.0,0.070453,0.069426,0.071496,10.214141,10.210438
4,1000069,7P/Pons-Winnecke,7P,NaN,NaN,JPL 24,2457096.5,57096,2015-03-15.0,0.6376,...,0,0.66645,24,2478421.074,46:00.0,0.194371,0.194358,0.194383,17.329157,17.328366
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11501,54232323,(2021 YX),2021 YX,N,25.23,JPL 1,2459578.5,59578,2021-12-30.0,0.5326,...,8,0.27999,2,2459584.751,02:00.0,0.025700,0.025422,0.025978,11.930361,11.921668
11502,54234943,(2022 AD),2022 AD,N,24.12,JPL 2,2459600.5,59600,2022-01-21.0,0.0586,...,6,0.45248,8,2476772.249,58:00.0,0.154193,0.112222,0.202105,7.875996,7.873802
11503,54235405,(2022 AF),2022 AF,N,23.54,JPL 1,2459583.5,59583,2022-01-04.0,0.4111,...,8,0.34340,9,2459586.805,20:00.0,0.062150,0.061829,0.062471,9.759260,9.754866
11504,54235406,(2022 AG),2022 AG,N,25.01,JPL 1,2459583.5,59583,2022-01-04.0,0.1730,...,8,0.49550,10,2484014.565,34:00.0,0.141628,0.119149,0.164802,7.980823,7.978465


In [4]:
### Performing Pre-processing on Future_df

In [5]:
# dropping rows with any null values
future_df = future_df.dropna()
future_df

,spkid,full_name,pdes,pha,h,orbit_id,epoch,epoch.mjd,epoch.cal,e,...,condition_code,rms,orbit_id_cad,jd,cd,dist,dist_min,dist_max,v_rel,v_inf
29,2000433,433 Eros (A898 PA),433,N,10.43,JPL 659,2459600.5,59600,2022-01-21.0,0.2227,...,0,0.29796,659,2485544.158,47:00.0,0.182464,0.182464,0.182464,5.974914,5.972469
30,2000887,887 Alinda (A918 AA),887,N,13.87,JPL 428,2459600.5,59600,2022-01-21.0,0.5705,...,0,0.39776,429,2483346.410,50:00.0,0.166325,0.166321,0.166329,11.268948,11.267526
31,2001221,1221 Amor (1932 EA1),1221,N,17.39,JPL 110,2459600.5,59600,2022-01-21.0,0.4358,...,0,0.45902,110,2487063.705,56:00.0,0.140528,0.140527,0.140528,7.564784,7.562277
32,2001566,1566 Icarus (1949 MA),1566,Y,16.35,JPL 126,2459600.5,59600,2022-01-21.0,0.8269,...,0,0.34741,126,2484581.797,08:00.0,0.043536,0.043529,0.043543,28.169786,28.167613
33,2001620,1620 Geographos (1951 RA),1620,Y,15.29,JPL 649,2459600.5,59600,2022-01-21.0,0.3354,...,0,0.34288,652,2486123.940,34:00.0,0.190024,0.190023,0.190025,16.694264,16.693424
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11501,54232323,(2021 YX),2021 YX,N,25.23,JPL 1,2459578.5,59578,2021-12-30.0,0.5326,...,8,0.27999,2,2459584.751,02:00.0,0.025700,0.025422,0.025978,11.930361,11.921668
11502,54234943,(2022 AD),2022 AD,N,24.12,JPL 2,2459600.5,59600,2022-01-21.0,0.0586,...,6,0.45248,8,2476772.249,58:00.0,0.154193,0.112222,0.202105,7.875996,7.873802
11503,54235405,(2022 AF),2022 AF,N,23.54,JPL 1,2459583.5,59583,2022-01-04.0,0.4111,...,8,0.34340,9,2459586.805,20:00.0,0.062150,0.061829,0.062471,9.759260,9.754866
11504,54235406,(2022 AG),2022 AG,N,25.01,JPL 1,2459583.5,59583,2022-01-04.0,0.1730,...,8,0.49550,10,2484014.565,34:00.0,0.141628,0.119149,0.164802,7.980823,7.978465


In [6]:
#getting a summary of null values for each column
future_df.isnull().sum()

spkid             0
full_name         0
pdes              0
pha               0
h                 0
orbit_id          0
epoch             0
epoch.mjd         0
epoch.cal         0
e                 0
a                 0
q                 0
i                 0
om                0
w                 0
ma                0
ad                0
n                 0
tp                0
tp_cal            0
per               0
per_y             0
moid              0
moid_ld           0
moid_jup          0
t_jup             0
sigma_e           0
sigma_a           0
sigma_q           0
sigma_i           0
sigma_om          0
sigma_w           0
sigma_ma          0
sigma_ad          0
sigma_n           0
sigma_tp          0
sigma_per         0
class             0
producer          0
data_arc          0
first_obs         0
last_obs          0
n_obs_used        0
condition_code    0
rms               0
orbit_id_cad      0
jd                0
cd                0
dist              0
dist_min          0


#### Define list of features for making future predictions based on trained neural networks features


In [7]:
# Define feature columns to subset dataframe
feature_columns = ["moid","moid_ld","h","v_rel","v_inf"]

In [8]:
output_df = future_df.loc[:, feature_columns]

In [9]:
output_df

,moid,moid_ld,h,v_rel,v_inf
29,0.149000,58.0000,10.43,5.974914,5.972469
30,0.081500,31.7000,13.87,11.268948,11.267526
31,0.106000,41.4000,17.39,7.564784,7.562277
32,0.034200,13.3000,16.35,28.169786,28.167613
33,0.030900,12.0000,15.29,16.694264,16.693424
...,...,...,...,...,...
11501,0.024300,9.4700,25.23,11.930361,11.921668
11502,0.061100,23.8000,24.12,7.875996,7.873802
11503,0.055200,21.5000,23.54,9.759260,9.754866
11504,0.000179,0.0698,25.01,7.980823,7.978465


In [10]:
# Extract feature variables from future data set
X = output_df
X

,moid,moid_ld,h,v_rel,v_inf
29,0.149000,58.0000,10.43,5.974914,5.972469
30,0.081500,31.7000,13.87,11.268948,11.267526
31,0.106000,41.4000,17.39,7.564784,7.562277
32,0.034200,13.3000,16.35,28.169786,28.167613
33,0.030900,12.0000,15.29,16.694264,16.693424
...,...,...,...,...,...
11501,0.024300,9.4700,25.23,11.930361,11.921668
11502,0.061100,23.8000,24.12,7.875996,7.873802
11503,0.055200,21.5000,23.54,9.759260,9.754866
11504,0.000179,0.0698,25.01,7.980823,7.978465


In [26]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X)

# Scale the data
X_scaled = X_scaler.transform(X)


In [11]:
# Load the Neural Networks Model "nn"
forecast_model = tf.keras.models.load_model('NEOs_project_NN.h5')


In [27]:
# Evaluate the forecast model using the future data
predictions = forecast_model.predict(X_scaled)

In [28]:
predictions

array([[0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       ...,
       [1.8665196e-25],
       [6.9712856e-29],
       [2.0613853e-13]], dtype=float32)

In [29]:
# making a data frame from predictions of future CAD and NEOs data
prediction_df = pd.DataFrame(predictions, columns = ["predicted_pha"])
prediction_df

,predicted_pha
0,0.000000e+00
1,0.000000e+00
2,0.000000e+00
3,9.994975e-01
4,9.990589e-01
...,...
11433,1.581736e-29
11434,0.000000e+00
11435,1.866520e-25
11436,6.971286e-29


In [ ]:
# transform predictions arrayt to 1 or 0 
def normalizer(prd):
    if prd > 0.5:
        return 1
    else:
        return 0

    
future_predicted_df["pha"] = future_predicted_df["pha"].apply(change)
future_predicted_df.head()

In [19]:
# merging prediction_df with future_df to incorporate our forecasts to the future
future_predicted_df = future_df.merge(prediction_df,left_index=True,right_index=True)
future_predicted_df

,spkid,full_name,pdes,pha,h,orbit_id,epoch,epoch.mjd,epoch.cal,e,...,rms,orbit_id_cad,jd,cd,dist,dist_min,dist_max,v_rel,v_inf,predicted_pha
29,2000433,433 Eros (A898 PA),433,N,10.43,JPL 659,2459600.5,59600,2022-01-21.0,0.2227,...,0.29796,659,2485544.158,47:00.0,0.182464,0.182464,0.182464,5.974914,5.972469,0.0
30,2000887,887 Alinda (A918 AA),887,N,13.87,JPL 428,2459600.5,59600,2022-01-21.0,0.5705,...,0.39776,429,2483346.410,50:00.0,0.166325,0.166321,0.166329,11.268948,11.267526,0.0
31,2001221,1221 Amor (1932 EA1),1221,N,17.39,JPL 110,2459600.5,59600,2022-01-21.0,0.4358,...,0.45902,110,2487063.705,56:00.0,0.140528,0.140527,0.140528,7.564784,7.562277,0.0
32,2001566,1566 Icarus (1949 MA),1566,Y,16.35,JPL 126,2459600.5,59600,2022-01-21.0,0.8269,...,0.34741,126,2484581.797,08:00.0,0.043536,0.043529,0.043543,28.169786,28.167613,0.0
33,2001620,1620 Geographos (1951 RA),1620,Y,15.29,JPL 649,2459600.5,59600,2022-01-21.0,0.3354,...,0.34288,652,2486123.940,34:00.0,0.190024,0.190023,0.190025,16.694264,16.693424,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11432,54225502,(2021 OY20),2021 OY20,N,19.66,JPL 9,2459600.5,59600,2022-01-21.0,0.3235,...,0.45502,10,2482175.931,21:00.0,0.080838,0.069741,0.136147,12.728090,12.725500,0.0
11433,54225504,(2021 PL132),2021 PL132,N,28.54,JPL 1,2459600.5,59600,2022-01-21.0,0.1582,...,0.45841,1,2471117.633,11:00.0,0.011404,0.002657,0.129844,12.502122,12.483419,0.0
11435,54225508,(2021 WR4),2021 WR4,N,30.41,JPL 2,2459600.5,59600,2022-01-21.0,0.2517,...,0.43439,2,2464379.501,02:00.0,0.130352,0.081906,0.178861,8.474801,8.472389,0.0
11436,54225509,(2021 WS4),2021 WS4,N,30.64,JPL 3,2459600.5,59600,2022-01-21.0,0.0991,...,0.43884,3,2486880.941,34:00.0,0.146996,0.109676,0.184944,5.775599,5.772460,0.0


In [20]:
# transform pha to 1 or 0 from object variables of Yes and No
def change(type):
    if type == "Y":
        return 1
    else:
        return 0
    
future_predicted_df["pha"] = future_predicted_df["pha"].apply(change)
future_predicted_df.head()

,spkid,full_name,pdes,pha,h,orbit_id,epoch,epoch.mjd,epoch.cal,e,...,rms,orbit_id_cad,jd,cd,dist,dist_min,dist_max,v_rel,v_inf,predicted_pha
29,2000433,433 Eros (A898 PA),433,0,10.43,JPL 659,2459600.5,59600,2022-01-21.0,0.2227,...,0.29796,659,2485544.158,47:00.0,0.182464,0.182464,0.182464,5.974914,5.972469,0.0
30,2000887,887 Alinda (A918 AA),887,0,13.87,JPL 428,2459600.5,59600,2022-01-21.0,0.5705,...,0.39776,429,2483346.410,50:00.0,0.166325,0.166321,0.166329,11.268948,11.267526,0.0
31,2001221,1221 Amor (1932 EA1),1221,0,17.39,JPL 110,2459600.5,59600,2022-01-21.0,0.4358,...,0.45902,110,2487063.705,56:00.0,0.140528,0.140527,0.140528,7.564784,7.562277,0.0
32,2001566,1566 Icarus (1949 MA),1566,1,16.35,JPL 126,2459600.5,59600,2022-01-21.0,0.8269,...,0.34741,126,2484581.797,08:00.0,0.043536,0.043529,0.043543,28.169786,28.167613,0.0
33,2001620,1620 Geographos (1951 RA),1620,1,15.29,JPL 649,2459600.5,59600,2022-01-21.0,0.3354,...,0.34288,652,2486123.940,34:00.0,0.190024,0.190023,0.190025,16.694264,16.693424,0.0


In [21]:
future_predicted_df['predicted_pha'].value_counts()

0.0    11370
Name: predicted_pha, dtype: int64

In [23]:
future_predicted_df['pha'].value_counts()

0    9522
1    1848
Name: pha, dtype: int64

In [ ]:
new_output_df.head()

In [ ]:
new_output_df.to_csv('test_future_predictions_data.csv', index=False)